In [336]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import plotly.graph_objs as go
from matplotlib import pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler

In [337]:
# Now, lets bring in our data
beer_df = pd.read_csv('my_beer.csv')

# I am going to display the data and our columns
print(beer_df.columns)

beer_df.head()

Index(['beer_name', 'brewery_name', 'beer_type', 'beer_abv', 'beer_ibu',
       'comment', 'venue_name', 'venue_city', 'venue_state', 'venue_country',
       'venue_lat', 'venue_lng', 'rating_score', 'created_at', 'checkin_url',
       'beer_url', 'brewery_url', 'brewery_country', 'brewery_city',
       'brewery_state', 'flavor_profiles', 'purchase_venue', 'serving_type',
       'checkin_id', 'bid', 'brewery_id', 'photo_url', 'global_rating_score',
       'global_weighted_rating_score', 'tagged_friends', 'total_toasts',
       'total_comments'],
      dtype='object')


,beer_name,brewery_name,beer_type,beer_abv,beer_ibu,comment,venue_name,venue_city,venue_state,venue_country,...,serving_type,checkin_id,bid,brewery_id,photo_url,global_rating_score,global_weighted_rating_score,tagged_friends,total_toasts,total_comments
0,Delirium Tremens,Delirium - Huyghe Brewery,Belgian Strong Golden Ale,8.5,24,A solid beer that would be easy to drink exclu...,NaN,NaN,NaN,NaN,...,Draft,921065799,4485,285,NaN,3.87,3.87,NaN,0,0
1,Delirium Nocturnum,Delirium - Huyghe Brewery,Belgian Strong Dark Ale,8.5,24,A darker beer from the Delirium brand. Would b...,NaN,NaN,NaN,NaN,...,Bottle,921066266,4486,285,NaN,3.78,3.78,NaN,0,0
2,La Guillotine,Delirium - Huyghe Brewery,Belgian Strong Golden Ale,8.5,0,"A very good brew. It is quite strong, which gi...",NaN,NaN,NaN,NaN,...,Draft,921066913,26907,285,NaN,3.58,3.58,NaN,0,0
3,Hanalei Island IPA,Kona Brewing Company,IPA - Session,4.5,40,NaN,NaN,NaN,NaN,NaN,...,Bottle,921067292,1604713,1988,NaN,3.55,3.55,NaN,0,0
4,Delirium Argentum,Delirium - Huyghe Brewery,IPA - Belgian,7.0,48,My favorite IPA to date. A very solid beer.,NaN,NaN,NaN,NaN,...,Draft,921067547,800757,285,NaN,3.61,3.61,NaN,0,0


In [338]:
# A lot of the columns we have are unnecessary, so lets drop a few
beer_df = beer_df.drop(['comment', 'venue_name', 'venue_city', 'venue_state', 'venue_country',
       'venue_lat', 'venue_lng', 'checkin_url', 'beer_url', 'brewery_url', 'purchase_venue',
       'checkin_id', 'photo_url', 'tagged_friends', 'total_toasts','total_comments', 'brewery_id', 'beer_ibu',
        'brewery_city', 'brewery_state', 'bid', 'global_weighted_rating_score'], axis=1)

beer_df.head()   

,beer_name,brewery_name,beer_type,beer_abv,rating_score,created_at,brewery_country,flavor_profiles,serving_type,global_rating_score
0,Delirium Tremens,Delirium - Huyghe Brewery,Belgian Strong Golden Ale,8.5,3.75,2020-07-23 18:01:01,Belgium,"light,smooth",Draft,3.87
1,Delirium Nocturnum,Delirium - Huyghe Brewery,Belgian Strong Dark Ale,8.5,3.00,2020-07-23 18:02:49,Belgium,"strong,dark",Bottle,3.78
2,La Guillotine,Delirium - Huyghe Brewery,Belgian Strong Golden Ale,8.5,4.00,2020-07-23 18:05:09,Belgium,"light,strong",Draft,3.58
3,Hanalei Island IPA,Kona Brewing Company,IPA - Session,4.5,4.00,2020-07-23 18:06:26,United States,"tropical,fruity,hoppy",Bottle,3.55
4,Delirium Argentum,Delirium - Huyghe Brewery,IPA - Belgian,7.0,4.00,2020-07-23 18:07:14,Belgium,"smooth,hoppy",Draft,3.61


In [339]:
# Now, I am going to rename my columns
beer_df = beer_df.rename(columns={'beer_name' : 'Beer Name', 'brewery_name' : 'Brewery Name', 'beer_type' : 'Beer Type', 'beer_abv' : 'Alcohol By Vol',
'rating_score' : 'My Rating', 'created_at' : 'Date of Rating', 'brewery_country' : 'Brewery Country','flavor_profiles' : 'Flavor Profiles', 'serving_type' : 'Serving Type',
'global_rating_score' : 'Global Rating'})

beer_df.head()

,Beer Name,Brewery Name,Beer Type,Alcohol By Vol,My Rating,Date of Rating,Brewery Country,Flavor Profiles,Serving Type,Global Rating
0,Delirium Tremens,Delirium - Huyghe Brewery,Belgian Strong Golden Ale,8.5,3.75,2020-07-23 18:01:01,Belgium,"light,smooth",Draft,3.87
1,Delirium Nocturnum,Delirium - Huyghe Brewery,Belgian Strong Dark Ale,8.5,3.00,2020-07-23 18:02:49,Belgium,"strong,dark",Bottle,3.78
2,La Guillotine,Delirium - Huyghe Brewery,Belgian Strong Golden Ale,8.5,4.00,2020-07-23 18:05:09,Belgium,"light,strong",Draft,3.58
3,Hanalei Island IPA,Kona Brewing Company,IPA - Session,4.5,4.00,2020-07-23 18:06:26,United States,"tropical,fruity,hoppy",Bottle,3.55
4,Delirium Argentum,Delirium - Huyghe Brewery,IPA - Belgian,7.0,4.00,2020-07-23 18:07:14,Belgium,"smooth,hoppy",Draft,3.61


In [340]:
# Now, I am going to format the Date of Rating to just the year
beer_df['Date of Rating'] = pd.to_datetime(beer_df['Date of Rating'])

beer_df['Date of Rating'] = beer_df['Date of Rating'].dt.strftime('%Y')

beer_df.head()

,Beer Name,Brewery Name,Beer Type,Alcohol By Vol,My Rating,Date of Rating,Brewery Country,Flavor Profiles,Serving Type,Global Rating
0,Delirium Tremens,Delirium - Huyghe Brewery,Belgian Strong Golden Ale,8.5,3.75,2020,Belgium,"light,smooth",Draft,3.87
1,Delirium Nocturnum,Delirium - Huyghe Brewery,Belgian Strong Dark Ale,8.5,3.00,2020,Belgium,"strong,dark",Bottle,3.78
2,La Guillotine,Delirium - Huyghe Brewery,Belgian Strong Golden Ale,8.5,4.00,2020,Belgium,"light,strong",Draft,3.58
3,Hanalei Island IPA,Kona Brewing Company,IPA - Session,4.5,4.00,2020,United States,"tropical,fruity,hoppy",Bottle,3.55
4,Delirium Argentum,Delirium - Huyghe Brewery,IPA - Belgian,7.0,4.00,2020,Belgium,"smooth,hoppy",Draft,3.61


In [341]:
# Let's drop the na values, as they will negatively impact our work down the line
beer_df = beer_df.dropna()

# Let's also drop the rows where abv == 0 if they exist
zero_min = beer_df.loc[beer_df['Alcohol By Vol'] == 0.0].index
beer_df = beer_df.drop(labels=zero_min, axis=0)

# Might come back to this and clean it up a bit later

# Now, let's check our work
# print(np.min(beer_df['Alcohol By Vol']))

In [342]:
# Let's get an idea of the "My Ratings" column vs the Global Ratings column

# Change the series to a type floar
beer_df['My Rating'] = round(beer_df['My Rating'].astype(float), 2)

# My average
average_rating = round(np.average(beer_df['My Rating']), 2)

# Global average
average_global_rating = round(np.average(beer_df['Global Rating']), 2)

# The difference in decimal and percentage
rating_difference = average_global_rating - average_rating

# Formatting String with the outcomes
print(f"""
Calculations for my rating vs. the global rating:

My average rating for my beer history is: {average_rating}, 
as opposed the the global average, which is: {average_global_rating}. 

The difference is: {round(rating_difference, 2)}, 
or a percentage of: {round(rating_difference * 100, 2)}%.
""")


Calculations for my rating vs. the global rating:

My average rating for my beer history is: 3.59, 
as opposed the the global average, which is: 3.7. 

The difference is: 0.11, 
or a percentage of: 11.0%.



In [343]:
# Let's now add ABV conditions
abv_numbers = [
  (beer_df['Alcohol By Vol'] >= 1.2) & (beer_df['Alcohol By Vol'] <= 4.9),
  (beer_df['Alcohol By Vol'] >= 5.0) & (beer_df['Alcohol By Vol'] <= 7.9),
  (beer_df['Alcohol By Vol'] >= 8.0) & (beer_df['Alcohol By Vol'] <= 15.0)
]

# Labels for the ABV types
abv_labels = ['Light', 'Medium', 'Strong']

# The new column with the ABV types
beer_df['ABV Type'] = np.select(abv_numbers, abv_labels)

# Let us ensure it worked
beer_df.head()

,Beer Name,Brewery Name,Beer Type,Alcohol By Vol,My Rating,Date of Rating,Brewery Country,Flavor Profiles,Serving Type,Global Rating,ABV Type
0,Delirium Tremens,Delirium - Huyghe Brewery,Belgian Strong Golden Ale,8.5,3.75,2020,Belgium,"light,smooth",Draft,3.87,Strong
1,Delirium Nocturnum,Delirium - Huyghe Brewery,Belgian Strong Dark Ale,8.5,3.00,2020,Belgium,"strong,dark",Bottle,3.78,Strong
2,La Guillotine,Delirium - Huyghe Brewery,Belgian Strong Golden Ale,8.5,4.00,2020,Belgium,"light,strong",Draft,3.58,Strong
3,Hanalei Island IPA,Kona Brewing Company,IPA - Session,4.5,4.00,2020,United States,"tropical,fruity,hoppy",Bottle,3.55,Light
4,Delirium Argentum,Delirium - Huyghe Brewery,IPA - Belgian,7.0,4.00,2020,Belgium,"smooth,hoppy",Draft,3.61,Medium


In [344]:
# Now, lets get some counts for how many Light, Medium, and Strong beers we have
light_abv = len(beer_df.loc[beer_df['ABV Type'] == 'Light'])
medium_abv = len(beer_df.loc[beer_df['ABV Type'] == 'Medium'])
strong_abv = len(beer_df.loc[beer_df['ABV Type'] == 'Strong'])

# Add them all together
total_beer = light_abv + medium_abv + strong_abv

# Now to calculate the percentages
light_abv_perc = round(light_abv/total_beer * 100, 2)
medium_abv_perc = round(medium_abv/total_beer * 100, 2)
strong_abv_perc = round(strong_abv/total_beer * 100, 2)

# Printing the number of entries
print(f"""
Total Count: {total_beer} brews

Count for Light Beer: {light_abv} brews, 
{light_abv_perc}% of the total count.

Count for Medium Beer: {medium_abv} brews, 
{medium_abv_perc}% of the total count.

Count for Strong Beer: {strong_abv} brews, 
{strong_abv_perc}% of the total count.
""")


Total Count: 233 brews

Count for Light Beer: 38 brews, 
16.31% of the total count.

Count for Medium Beer: 148 brews, 
63.52% of the total count.

Count for Strong Beer: 47 brews, 
20.17% of the total count.



In [345]:
# Now, lets also see the percentage for which year I drank the most beer

# First, since our column is a datetime type, we need to make it an int
beer_df['Date of Rating'] = beer_df['Date of Rating'].astype(int)

# Lets get all of the counts for each year
beer_2020 = len(beer_df.loc[beer_df['Date of Rating'] == 2020])
beer_2021 = len(beer_df.loc[beer_df['Date of Rating'] == 2021])
beer_2022 = len(beer_df.loc[beer_df['Date of Rating'] == 2022])

# Add them up
total_beer_per_year = beer_2020 + beer_2021 + beer_2022

# # Now to calculate the percentages
beer_2020_perc = round(beer_2020/total_beer_per_year * 100, 2)
beer_2021_perc = round(beer_2021/total_beer_per_year * 100, 2)
beer_2022_perc = round(beer_2022/total_beer_per_year * 100, 2)

# Printing the number of entries via len()
print(f"""
Total count: {total_beer_per_year} brews

Count for 2020: {beer_2020} brews, 
{beer_2020_perc}% of the total count.

Count for 2021: {beer_2021} brews, 
{beer_2021_perc}% of the total count.

Count for 2022: {beer_2022} brews, 
{beer_2022_perc}% of the total count.
""")


Total count: 233 brews

Count for 2020: 68 brews, 
29.18% of the total count.

Count for 2021: 102 brews, 
43.78% of the total count.

Count for 2022: 63 brews, 
27.04% of the total count.



In [346]:
# I am also interested to see which serving type I liked the most. For this calculation, I am going to make a new frame tailored to what I need
serving_type_frame = beer_df[['My Rating', 'Global Rating' ,'Serving Type']]

serving_type_frame.head()

,My Rating,Global Rating,Serving Type
0,3.75,3.87,Draft
1,3.00,3.78,Bottle
2,4.00,3.58,Draft
3,4.00,3.55,Bottle
4,4.00,3.61,Draft


In [347]:
# Now, lets see all of our unique values in the Serving Type column
unique_serving_type = serving_type_frame['Serving Type'].unique()
print(unique_serving_type)

['Draft' 'Bottle' 'Can']


In [348]:
# We have 3 unique values. So, lets go ahead and extract those values
serving_type_bottle = serving_type_frame.loc[serving_type_frame['Serving Type'] == 'Bottle']
serving_type_can = serving_type_frame.loc[serving_type_frame['Serving Type'] == 'Can']
serving_type_draft = serving_type_frame.loc[serving_type_frame['Serving Type'] == 'Draft']

# Just as a curiosity, lets follow a calculation like we did for years and abv, so, I'll add them all
total_serving_type = len(serving_type_bottle) + len(serving_type_can) + len(serving_type_draft)

# Get the calculations
serv_ty_bot_perc = round(len(serving_type_bottle) / total_serving_type * 100, 2)
serv_ty_can_perc = round(len(serving_type_can) / total_serving_type * 100, 2)
serv_ty_draft_perc = round(len(serving_type_draft) / total_serving_type * 100, 2)

# Lets now see what we came up with
print(f"""
Total count: {total_serving_type} brews

Count for Bottles: {len(serving_type_bottle)} brews, 
{serv_ty_bot_perc}% of the total count.

Count for Cans: {len(serving_type_can)} brews, 
{serv_ty_can_perc}% of the total count.

Count for Draft: {len(serving_type_draft)} brews, 
{serv_ty_draft_perc}% of the total count.
""")


Total count: 233 brews

Count for Bottles: 68 brews, 
29.18% of the total count.

Count for Cans: 43 brews, 
18.45% of the total count.

Count for Draft: 122 brews, 
52.36% of the total count.



In [349]:
# We had those variable for serving types and the serving_type_frame for a reason. Lets start with some basic calculations I may use for visualizing
serving_type_bottle_avg = round(np.average(serving_type_bottle['My Rating']), 2)
serving_type_can_avg = round(np.average(serving_type_can['My Rating']), 2)
serving_type_draft_avg = round(np.average(serving_type_draft['My Rating']), 2)
print(f"""
Average Rating for Bottles: {serving_type_bottle_avg}/5

Average Rating for Cans: {serving_type_can_avg}/5

Average Rating for Draft: {serving_type_draft_avg}/5
""")


Average Rating for Bottles: 3.47/5

Average Rating for Cans: 3.58/5

Average Rating for Draft: 3.67/5



In [350]:
# Copy the frame
scaled_beer_frame = beer_df.copy()

# Isolation forest model
model = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', max_features=1.0)
model.fit(scaled_beer_frame[['Alcohol By Vol', 'My Rating', 'Global Rating']])

# Scores and anomaly detection
scaled_beer_frame['ABV Scores'] = model.decision_function(scaled_beer_frame[['Alcohol By Vol', 'My Rating', 'Global Rating']])
scaled_beer_frame['Anomalies'] = model.predict(scaled_beer_frame[['Alcohol By Vol', 'My Rating', 'Global Rating']])

In [351]:
# Now, we can also go ahead and apply the anomalous column to our original frame
beer_df['Anomalies'] = scaled_beer_frame['Anomalies']

# Now, lets get and idea of how many anomalies 
anoms = beer_df.loc[beer_df['Anomalies'] == -1]
print(len(anoms))

# Lets also get the index values, we can use them later
anom_index = list(anoms.index)

# And now, we can see exactly which ones are anomalous (commenting out because the list is rather extensive)
# anoms.loc[anom_index]

# Going to use anoms for visualizing
anoms.head()

37


,Beer Name,Brewery Name,Beer Type,Alcohol By Vol,My Rating,Date of Rating,Brewery Country,Flavor Profiles,Serving Type,Global Rating,ABV Type,Anomalies
6,Trappist Westvleteren 12,Brouwerij De Sint-Sixtusabdij van Westvleteren,Belgian Quadrupel,10.2,3.50,2020,Belgium,"rich,smooth,dark",Bottle,4.51,Strong,-1
8,Gruut Inferno,Gentse Gruut Stadsbrouwerij,Belgian Tripel,9.0,4.25,2020,Belgium,"strong,bitter",Bottle,3.53,Strong,-1
17,Beck's,Brauerei Beck,Pilsner - German,4.9,2.50,2020,Germany,"clean,light",Bottle,2.95,Light,-1
22,Delirium Black Barrel Aged,Delirium - Huyghe Brewery,Belgian Strong Dark Ale,11.5,4.00,2020,Belgium,"barrel aged,bourbon",Bottle,4.06,Strong,-1
28,Préaris Grand Cru 2018 Laphroaig,Vliegende Paard Brouwers,Belgian Quadrupel,10.0,4.50,2020,Belgium,"full,peaty",Bottle,3.88,Strong,-1


In [352]:
# Time for a visualization! This is Anomalies (vals of -1 in Anomalies Column) by My Rating
chart_data = pd.concat([
	anoms['My Rating'],
	anoms['Anomalies'],
], axis=1)
chart_data = chart_data.sort_values(['My Rating'])
chart_data = chart_data.rename(columns={'My Rating': 'x'})
chart_data_count = chart_data.groupby(['x'])[['Anomalies']].count()
chart_data_count.columns = ['Anomalies|count']
chart_data = chart_data_count.reset_index()
chart_data = chart_data.dropna()

charts = []
charts.append(go.Bar(
	x=chart_data['x'],
	y=chart_data['Anomalies|count']
))
figure = go.Figure(data=charts, layout=go.Layout({
    'barmode': 'group',
    'legend': {'orientation': 'h'},
    'title': {'text': 'Count of Anomalies by My Rating'},
    'xaxis': {'title': {'text': 'My Rating'}},
    'yaxis': {'tickformat': '0:g', 'title': {'text': 'Count of Anomalies'}, 'type': 'linear'}
}))

figure.show()